![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Stage 1: Install dependencies and setting up GPU environment

In [1]:
!pip install tqdm

### Downloading the Dogs vs Cats dataset 

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-01-19 13:18:22--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.206.128, 74.125.133.128, 74.125.140.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.206.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  58.6MB/s    in 1.1s    

2021-01-19 13:18:23 (58.6 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import project dependencies

In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.4.0'

### Unzipping the Dogs vs Cats dataset

In [4]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [5]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [6]:
zip_object.extractall("./")

In [7]:
zip_object.close()

### Seting up dataset paths

In [8]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [9]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [10]:
IMG_SHAPE = (128, 128, 3)

In [11]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [12]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [13]:
len(base_model.layers)

154

### Freezing the base model

In [14]:
base_model.trainable = False

### Defining the custom head for our network

In [15]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [18]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [19]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [21]:
len(model.layers)

156

### Compiling the model

In [22]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [23]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [24]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [25]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [26]:
%%time
model.fit_generator(train_generator, epochs=5, steps_per_epoch=len(train_generator), validation_data=valid_generator, validation_steps=len(valid_generator))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 20s 721ms/step - loss: 0.7363 - accuracy: 0.5482 - val_loss: 0.6207 - val_accuracy: 0.6670
Epoch 2/5
16/16 [==============================] - 9s 609ms/step - loss: 0.6096 - accuracy: 0.6642 - val_loss: 0.5385 - val_accuracy: 0.7480
Epoch 3/5
16/16 [==============================] - 9s 597ms/step - loss: 0.5273 - accuracy: 0.7390 - val_loss: 0.4716 - val_accuracy: 0.8010
Epoch 4/5
16/16 [==============================] - 9s 590ms/step - loss: 0.4600 - accuracy: 0.8078 - val_loss: 0.4153 - val_accuracy: 0.8410
Epoch 5/5
16/16 [==============================] - 9s 585ms/step - loss: 0.4068 - accuracy: 0.8407 - val_loss: 0.3680 - val_accuracy: 0.8730
CPU times: user 54.9 s, sys: 2.49 s, total: 57.4 s
Wall time: 58.1 s


### Transfer learning model evaluation

In [27]:
valid_loss, valid_accuracy = model.evaluate(valid_generator, batch_size = len(valid_generator), steps = len(valid_generator))

8/8 [==============================] - 3s 379ms/step - loss: 0.3680 - accuracy: 0.8730


In [28]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8730000257492065


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [29]:
base_model.trainable = True

In [30]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [31]:
fine_tune_at = 100

In [32]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [33]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [34]:
%%time
model.fit_generator(train_generator, epochs=5, steps_per_epoch=len(train_generator), validation_data=valid_generator, validation_steps=len(valid_generator))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 16s 680ms/step - loss: 0.2637 - accuracy: 0.8879 - val_loss: 0.0860 - val_accuracy: 0.9690
Epoch 2/5
16/16 [==============================] - 9s 596ms/step - loss: 0.0302 - accuracy: 0.9963 - val_loss: 0.1091 - val_accuracy: 0.9660
Epoch 3/5
16/16 [==============================] - 9s 594ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.1085 - val_accuracy: 0.9690
Epoch 4/5
16/16 [==============================] - 9s 594ms/step - loss: 0.0076 - accuracy: 0.9995 - val_loss: 0.1473 - val_accuracy: 0.9620
Epoch 5/5
16/16 [==============================] - 10s 603ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1855 - val_accuracy: 0.9570
CPU times: user 58.6 s, sys: 2.79 s, total: 1min 1s
Wall time: 54.6 s


### Evaluating the fine tuned model

In [35]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator, steps = len(valid_generator))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [36]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9570000171661377
